# Выбор локации для скважины

Являясь сотрудником добывающей компании «ГлавРосГосНефть», нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Необходимо проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

In [4]:
data1 = pd.read_csv('/datasets/gold_recovery_train_new.csv')
data1.head()

,date,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,...,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
0,2016-01-15 00:00:00,6.055403,9.889648,5.507324,42.192020,70.541216,10.411962,0.895447,16.904297,2.143149,...,14.016835,-502.488007,12.099931,-504.715942,9.925633,-498.310211,8.079666,-500.470978,14.151341,-605.841980
1,2016-01-15 01:00:00,6.029369,9.968944,5.257781,42.701629,69.266198,10.462676,0.927452,16.634514,2.224930,...,13.992281,-505.503262,11.950531,-501.331529,10.039245,-500.169983,7.984757,-500.582168,13.998353,-599.787184
2,2016-01-15 02:00:00,6.055926,10.213995,5.383759,42.657501,68.116445,10.507046,0.953716,16.208849,2.257889,...,14.015015,-502.520901,11.912783,-501.133383,10.070913,-500.129135,8.013877,-500.517572,14.028663,-601.427363
3,2016-01-15 03:00:00,6.047977,9.977019,4.858634,42.689819,68.347543,10.422762,0.883763,16.532835,2.146849,...,14.036510,-500.857308,11.999550,-501.193686,9.970366,-499.201640,7.977324,-500.255908,14.005551,-599.996129
4,2016-01-15 04:00:00,6.148599,10.142511,4.939416,42.774141,66.927016,10.360302,0.792826,16.525686,2.055292,...,14.027298,-499.838632,11.953070,-501.053894,9.925709,-501.686727,7.894242,-500.356035,13.996647,-601.496691


In [3]:
data2 = pd.read_csv('/datasets/geo_data_1.csv')
data2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [4]:
data3 = pd.read_csv('/datasets/geo_data_2.csv')
data3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [5]:
data1.info()
data2.info()
data3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

Посмотрим, есть ли дубликаты:

In [6]:
data1[data1.duplicated()==True]

,id,f0,f1,f2,product


In [7]:
data2[data2.duplicated()==True]

,id,f0,f1,f2,product


In [8]:
data3[data3.duplicated()==True]

,id,f0,f1,f2,product


Дубликатов нет, данные чистые, без пропусков, имеют рабочий тип данных, поэтому, можно приступать к анализу.

In [9]:
data1[data1.duplicated('id')]

,id,f0,f1,f2,product
7530,HZww2,1.061194,-0.373969,10.430210,158.828695
41724,bxg6G,-0.823752,0.546319,3.630479,93.007798
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205
63593,QcMuo,0.635635,-0.473422,0.862670,64.578675
66136,74z30,1.084962,-0.312358,6.990771,127.643327
69163,AGS9W,-0.933795,0.116194,-3.655896,19.230453
75715,Tdehs,0.112079,0.430296,3.218993,60.964018
90815,fiKDv,0.049883,0.841313,6.394613,137.346586
92341,TtcGQ,0.110711,1.022689,0.911381,101.318008
97785,bsk9y,0.378429,0.005837,0.160827,160.637302


In [10]:
data1[data1['id']=='HZww2']

,id,f0,f1,f2,product
931,HZww2,0.755284,0.368511,1.863211,30.681774
7530,HZww2,1.061194,-0.373969,10.430210,158.828695


In [11]:
data2[data2.duplicated('id')]

,id,f0,f1,f2,product
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044


In [12]:
data3[data3.duplicated('id')]

,id,f0,f1,f2,product
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420
55967,KUPhW,1.211150,3.176408,5.543540,132.831802
95090,Vcm5J,2.587702,1.986875,2.482245,92.327572


data1.plot(x='id', y='product', kind='scatter', grid=True, alpha=0.03)
data1['product'].hist(bins = 10, figsize = (15,3), range = (0,250))

## Обучение и проверка модели

Удалим ненужный столбец для исследования id и разделим каждый датафрейм на признаки и целевые признаки:

In [13]:
data1.drop(columns = ['id'],axis = 1, inplace=True)
data2.drop(columns = ['id'],axis = 1, inplace=True)
data3.drop(columns = ['id'],axis = 1, inplace=True)

features1 = data1.drop(['product'], axis=1)
target1 = data1['product']
features_train1, features_valid1, target_train1, target_valid1 = train_test_split(\
    features1, target1, test_size=0.25, random_state=12345)

features2 = data2.drop(['product'], axis=1)
target2 = data2['product']
features_train2, features_valid2, target_train2, target_valid2 = train_test_split(\
    features2, target2, test_size=0.25, random_state=12345)

features3 = data3.drop(['product'], axis=1)
target3 = data3['product']
features_train3, features_valid3, target_train3, target_valid3 = train_test_split(\
    features3, target3, test_size=0.25, random_state=12345)

Обучим модели LinearRegression и сделаем предсказания:

In [14]:
model = LinearRegression()
model.fit(features_train1, target_train1)
predictions_valid1 = model.predict(features_valid1)
result1 = mean_squared_error(target_valid1, predictions_valid1) ** 0.5
predictions_mean1 = predictions_valid1.mean()
print("Cредний запас предсказанного сырья:", predictions_mean1)
print("RMSE модели линейной регрессии на валидационной выборке:", result1)

Cредний запас предсказанного сырья: 92.59256778438035
RMSE модели линейной регрессии на валидационной выборке: 37.5794217150813


In [15]:
model.fit(features_train2, target_train2)
predictions_valid2 = model.predict(features_valid2)
result2 = mean_squared_error(target_valid2, predictions_valid2) ** 0.5
predictions_mean2 = predictions_valid2.mean()
print("Cредний запас предсказанного сырья:", predictions_mean2)
print("RMSE модели линейной регрессии на валидационной выборке:", result2)

Cредний запас предсказанного сырья: 68.728546895446
RMSE модели линейной регрессии на валидационной выборке: 0.893099286775617


In [16]:
model.fit(features_train3, target_train3)
predictions_valid3 = model.predict(features_valid3)
result3 = mean_squared_error(target_valid3, predictions_valid3) ** 0.5
predictions_mean3 = predictions_valid3.mean()
print("Cредний запас предсказанного сырья:", predictions_mean3)
print("RMSE модели линейной регрессии на валидационной выборке:", result3)

Cредний запас предсказанного сырья: 94.96504596800489
RMSE модели линейной регрессии на валидационной выборке: 40.02970873393434


Cредний запас предсказанного сырья наиболее высокий у моделей 1 и 3, во второй модели гораздо ниже. Но RMSE второй модели самая низкая, что говорит о том, что кол-во ошибок достаточно мало.

## Подготовка к расчёту прибыли

Выпишем данные из условия задачи и сохраним в переменные:

In [17]:
BUDGET = 10_000_000_000 #Бюджет на разработку скважин в регионе — 10 млрд рублей.
PRICE_BARREL = 450_000 #При нынешних ценах один баррель сырья приносит 450 рублей дохода. 
#Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
VOLUME_BARREL = BUDGET / PRICE_BARREL / 200 #объем добычи, необходимый для безубыточного производства в регионе
print(VOLUME_BARREL)

111.11111111111111


Итак, для того, чтобы освоить бюджет, необходимо добывать не менее 111 тыс баррелей c одной точки. Средний запас сырья на 1 точке, рассчитанный выше для трех моделей: 93, 69 и 95 тыс баррелей. Соответственно, по 1 и 3 точке средние значения более близки к этому значению, но недостаточны. Логичнее подобрать точки с самими высокими запасами и работатать на них.

In [18]:
data1['product'].mean()

92.50000000000001

In [19]:
data2['product'].mean()

68.82500000000002

In [20]:
data3['product'].mean()

95.00000000000004

## Расчёт прибыли и рисков 

Создадим функцию по расчету прибыли:

In [21]:
def profit(target, probabilities, count):
    probs_sorted = pd.Series(probabilities, index=target.index).sort_values(ascending=False)
    target_sorted = target[probs_sorted.index]
    summa = target_sorted[:count].sum()
    profit_region = (summa * PRICE_BARREL) - BUDGET
    return profit_region

Проведем исследование средней прибыли, рисков по каждому отдельно региону:

In [22]:
state = np.random.RandomState(12345)
predictions_valid1 = pd.Series(predictions_valid1, index=target_valid1.index)
values = []
for i in range(1000):
    target_subsample = target_valid1.sample(n=500, replace=True, random_state=state)
    probs_subsample = predictions_valid1[target_subsample.index]
    values.append(profit(target_subsample, probs_subsample, count=200))

values = pd.Series(values)

lost=0  
for i in values:
    if i < 0:
        lost +=1
lower = values.quantile(0.025)
upper = values.quantile(0.975)
mean = values.mean()
risk = lost/1000
print("Данные по первому региону:")
print("Средняя прибыль:", round(mean))
print('Доверительный интервал:[',round(lower),':', round(upper),']')
print('Риск:', risk)

Данные по первому региону:
Средняя прибыль: 425938527
Доверительный интервал:[ -102090095 : 947976353 ]
Риск: 0.06


In [23]:
predictions_valid2 = pd.Series(predictions_valid2, index=target_valid2.index)
values = []
for i in range(1000):
    target_subsample = target_valid2.sample(n=500, replace=True, random_state=state)
    probs_subsample = predictions_valid2[target_subsample.index]
    values.append(profit(target_subsample, probs_subsample, count=200))

values = pd.Series(values)

lost=0  
for i in values:
    if i < 0:
        lost +=1
lower = values.quantile(0.025)
upper = values.quantile(0.975)
mean = values.mean()
risk = lost/1000
print("Данные по второму региону:")
print("Средняя прибыль:", round(mean))
print('Доверительный интервал:[',round(lower),':', round(upper),']')
print('Риск:', risk)

Данные по второму региону:
Средняя прибыль: 518259494
Доверительный интервал:[ 128123231 : 953612982 ]
Риск: 0.003


In [24]:
predictions_valid3 = pd.Series(predictions_valid3, index=target_valid3.index)
values = []
for i in range(1000):
    target_subsample = target_valid3.sample(n=500, replace=True, random_state=state)
    probs_subsample = predictions_valid3[target_subsample.index]
    values.append(profit(target_subsample, probs_subsample, count=200))

values = pd.Series(values)

lost=0  
for i in values:
    if i < 0:
        lost +=1
lower = values.quantile(0.025)
upper = values.quantile(0.975)
mean = values.mean()
risk = lost/1000
print("Данные по третьему региону:")
print("Средняя прибыль:", round(mean))
print('Доверительный интервал:[',round(lower),':', round(upper),']')
print('Риск:', risk)

Данные по третьему региону:
Средняя прибыль: 420194005
Доверительный интервал:[ -115852609 : 989629940 ]
Риск: 0.062


Как видим, самая высокая прибыль по 2 региону 518 млн.руб. и ниже риски, чем по остальным регионам, поэтому целесообразно выбрать для добычи 2 регион.